In [1]:
!pip install mlflow

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import mlflow
import mlflow.tensorflow

# Activer l'autolog MLflow pour TensorFlow/Keras

In [3]:
mlflow.tensorflow.autolog()

# Charger les fichiers d'images

In [4]:
dataset_path = "/content/drive/MyDrive/UTKFace"
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(".jpg")]
ages = [int(os.path.basename(f).split("_")[0]) for f in image_files]

# Séparer en train/validation (80% / 20%)

In [5]:
train_files, val_files, train_ages, val_ages = train_test_split(image_files, ages, test_size=0.2, random_state=42)

# Prétraitement des images

In [6]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128)) / 255.0  # Normalisation
    return img

# Générateur d'images avec augmentation

In [7]:
train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=20)
val_datagen = ImageDataGenerator()

In [8]:
def data_generator(file_paths, ages, batch_size=32, datagen=None):
    while True:
        if len(file_paths) == 0:
            raise ValueError("Le fichier d'entraînement est vide ! Vérifie train_files.")

        indices = np.random.choice(len(file_paths), batch_size, replace=False)
        batch_images = []
        batch_ages = []

        for i in indices:
            img = load_and_preprocess_image(file_paths[i])
            batch_images.append(img)
            batch_ages.append(ages[i])

        batch_images = np.array(batch_images)
        batch_ages = np.array(batch_ages)

        yield batch_images, batch_ages


# Définition du modèle TinyVGG

In [9]:
def build_tinyvgg(input_shape=(128, 128, 3)):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='linear')  # Régression d'âge
    ])
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])  # MSE pour régression
    return model

# Initialiser le modèle

In [10]:
model = build_tinyvgg()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       8,388,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,482,369 (32.36 MB)

 Trainable params: 8,482,369 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

# Entraînement avec MLflow

In [12]:
batch_size = 64
train_steps = len(train_files) // batch_size
val_steps = len(val_files) // batch_size

with mlflow.start_run():
    history = model.fit(
        data_generator(train_files, train_ages, batch_size, train_datagen),
        steps_per_epoch=train_steps,
        validation_data=data_generator(val_files, val_ages, batch_size, val_datagen),
        validation_steps=val_steps,
        epochs=30,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        ]
    )

    # Sauvegarde du modèle dans MLflow
    mlflow.tensorflow.log_model(model, "tinyvgg_model")


2025/02/08 15:27:31 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'generator'>. Dataset logging skipped.
2025/02/08 15:27:31 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'generator'>. Dataset logging skipped.


Epoch 1/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 10.8168 - mae: 10.8168

223/223 ━━━━━━━━━━━━━━━━━━━━ 681s 3s/step - loss: 10.8102 - mae: 10.8102 - val_loss: 6.5955 - val_mae: 6.5955
Epoch 2/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 7.4141 - mae: 7.4141

223/223 ━━━━━━━━━━━━━━━━━━━━ 683s 3s/step - loss: 7.4131 - mae: 7.4131 - val_loss: 5.9176 - val_mae: 5.9176
Epoch 3/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 6.6612 - mae: 6.6612

223/223 ━━━━━━━━━━━━━━━━━━━━ 681s 3s/step - loss: 6.6608 - mae: 6.6608 - val_loss: 5.7487 - val_mae: 5.7487
Epoch 4/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 6.2511 - mae: 6.2511

...

223/223 ━━━━━━━━━━━━━━━━━━━━ 646s 3s/step - loss: 5.6414 - mae: 5.6414 - val_loss: 5.4344 - val_mae: 5.4344
Epoch 8/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 5.5846 - mae: 5.5846

223/223 ━━━━━━━━━━━━━━━━━━━━ 648s 3s/step - loss: 5.5842 - mae: 5.5842 - val_loss: 4.9666 - val_mae: 4.9666
Epoch 9/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 9668s 44s/step - loss: 5.4070 - mae: 5.4070 - val_loss: 5.1768 - val_mae: 5.1768
Epoch 10/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 641s 3s/step - loss: 5.2277 - mae: 5.2277 - val_loss: 5.0903 - val_mae: 5.0903
Epoch 11/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - loss: 4.9752 - mae: 4.9752 

223/223 ━━━━━━━━━━━━━━━━━━━━ 3059s 14s/step - loss: 4.9754 - mae: 4.9754 - val_loss: 4.8779 - val_mae: 4.8779
Epoch 12/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 679s 3s/step - loss: 4.9339 - mae: 4.9339 - val_loss: 5.3020 - val_mae: 5.3020
Epoch 13/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 680s 3s/step - loss: 4.8836 - mae: 4.8836 - val_loss: 5.2609 - val_mae: 5.2609
Epoch 14/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.7773 - mae: 4.7773

223/223 ━━━━━━━━━━━━━━━━━━━━ 646s 3s/step - loss: 4.7772 - mae: 4.7772 - val_loss: 4.8226 - val_mae: 4.8226
Epoch 15/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 641s 3s/step - loss: 4.6900 - mae: 4.6900 - val_loss: 5.2001 - val_mae: 5.2001
Epoch 16/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 685s 3s/step - loss: 4.6558 - mae: 4.6558 - val_loss: 4.9629 - val_mae: 4.9629
Epoch 17/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 2599s 12s/step - loss: 4.5182 - mae: 4.5182 - val_loss: 5.0292 - val_mae: 5.0292


2025/02/08 22:19:01 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'generator'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2025/02/08 22:19:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/08 22:19:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/08 22:19:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core

# Fonction de prédiction et affichage

In [13]:
def predict_and_show_image(image_path):
    img = load_and_preprocess_image(image_path)
    img_batch = np.expand_dims(img, axis=0)

    predicted_age = model.predict(img_batch)[0][0]
    image_name = os.path.basename(image_path)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{image_name}\nÂge prédit : {predicted_age:.2f} ans")
    plt.show()

# Test avec une image UTKFace

In [ ]:
test_image = "/content/IMG_0392.jpeg"
predict_and_show_image(test_image)

In [14]:
from google.colab import files
files.download('/content/mlruns')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>